# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

# Eager mode on

In [2]:
tf.enable_eager_execution()

# Checkpoint function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

# Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.3464, train_loss: 2.08131313, train_accuracy: 0.3750, test_Accuracy: 0.2719
Epoch: [ 0] [    1/  468] time: 0.4808, train_loss: 2.02861357, train_accuracy: 0.4766, test_Accuracy: 0.4322
Epoch: [ 0] [    2/  468] time: 0.5887, train_loss: 1.91424179, train_accuracy: 0.5234, test_Accuracy: 0.5423
Epoch: [ 0] [    3/  468] time: 0.6909, train_loss: 1.76309013, train_accuracy: 0.6250, test_Accuracy: 0.5716
Epoch: [ 0] [    4/  468] time: 0.8009, train_loss: 1.71893096, train_accuracy: 0.5781, test_Accuracy: 0.6158
Epoch: [ 0] [    5/  468] time: 0.9101, train_loss: 1.68493593, train_accuracy: 0.6172, test_Accuracy: 0.6568
Epoch: [ 0] [    6/  468] time: 1.0192, train_loss: 1.45470881, train_accuracy: 0.6953, test_Accuracy: 0.6954
Epoch: [ 0] [    7/  468] time: 1.1196, train_loss: 1.30661106, train_accuracy: 0.7188, test_Accuracy: 0.7245
Epoch: [ 0] [    8/  468] time: 1.2275,

Epoch: [ 0] [   74/  468] time: 7.4990, train_loss: 0.31315297, train_accuracy: 0.8750, test_Accuracy: 0.9070
Epoch: [ 0] [   75/  468] time: 7.5926, train_loss: 0.24490650, train_accuracy: 0.9141, test_Accuracy: 0.9129
Epoch: [ 0] [   76/  468] time: 7.6862, train_loss: 0.41736907, train_accuracy: 0.8984, test_Accuracy: 0.9096
Epoch: [ 0] [   77/  468] time: 7.7752, train_loss: 0.37905023, train_accuracy: 0.8750, test_Accuracy: 0.9039
Epoch: [ 0] [   78/  468] time: 7.8668, train_loss: 0.39331663, train_accuracy: 0.8984, test_Accuracy: 0.9031
Epoch: [ 0] [   79/  468] time: 7.9565, train_loss: 0.34171745, train_accuracy: 0.8906, test_Accuracy: 0.9062
Epoch: [ 0] [   80/  468] time: 8.0505, train_loss: 0.33353072, train_accuracy: 0.9062, test_Accuracy: 0.9071
Epoch: [ 0] [   81/  468] time: 8.1385, train_loss: 0.28421435, train_accuracy: 0.9219, test_Accuracy: 0.9117
Epoch: [ 0] [   82/  468] time: 8.2317, train_loss: 0.30892992, train_accuracy: 0.9062, test_Accuracy: 0.9177
Epoch: [ 0

Epoch: [ 0] [  149/  468] time: 14.4213, train_loss: 0.15053803, train_accuracy: 0.9609, test_Accuracy: 0.9304
Epoch: [ 0] [  150/  468] time: 14.5153, train_loss: 0.26566446, train_accuracy: 0.9297, test_Accuracy: 0.9297
Epoch: [ 0] [  151/  468] time: 14.6067, train_loss: 0.19817568, train_accuracy: 0.9609, test_Accuracy: 0.9319
Epoch: [ 0] [  152/  468] time: 14.7006, train_loss: 0.22187383, train_accuracy: 0.9219, test_Accuracy: 0.9330
Epoch: [ 0] [  153/  468] time: 14.7871, train_loss: 0.25859040, train_accuracy: 0.9375, test_Accuracy: 0.9347
Epoch: [ 0] [  154/  468] time: 14.8762, train_loss: 0.16941282, train_accuracy: 0.9766, test_Accuracy: 0.9358
Epoch: [ 0] [  155/  468] time: 14.9704, train_loss: 0.15483929, train_accuracy: 0.9375, test_Accuracy: 0.9334
Epoch: [ 0] [  156/  468] time: 15.0619, train_loss: 0.14636864, train_accuracy: 0.9609, test_Accuracy: 0.9324
Epoch: [ 0] [  157/  468] time: 15.1600, train_loss: 0.28390071, train_accuracy: 0.8984, test_Accuracy: 0.9310
E

Epoch: [ 0] [  223/  468] time: 21.3319, train_loss: 0.26018322, train_accuracy: 0.9375, test_Accuracy: 0.9461
Epoch: [ 0] [  224/  468] time: 21.4210, train_loss: 0.18056758, train_accuracy: 0.9297, test_Accuracy: 0.9458
Epoch: [ 0] [  225/  468] time: 21.5114, train_loss: 0.21827528, train_accuracy: 0.9297, test_Accuracy: 0.9451
Epoch: [ 0] [  226/  468] time: 21.6023, train_loss: 0.11887397, train_accuracy: 0.9844, test_Accuracy: 0.9434
Epoch: [ 0] [  227/  468] time: 21.6993, train_loss: 0.20365638, train_accuracy: 0.9297, test_Accuracy: 0.9411
Epoch: [ 0] [  228/  468] time: 21.7960, train_loss: 0.29492781, train_accuracy: 0.9453, test_Accuracy: 0.9407
Epoch: [ 0] [  229/  468] time: 21.8949, train_loss: 0.19285861, train_accuracy: 0.9531, test_Accuracy: 0.9413
Epoch: [ 0] [  230/  468] time: 21.9877, train_loss: 0.28439391, train_accuracy: 0.9375, test_Accuracy: 0.9431
Epoch: [ 0] [  231/  468] time: 22.0838, train_loss: 0.13235317, train_accuracy: 0.9688, test_Accuracy: 0.9448
E

Epoch: [ 0] [  297/  468] time: 28.3874, train_loss: 0.14165691, train_accuracy: 0.9688, test_Accuracy: 0.9513
Epoch: [ 0] [  298/  468] time: 28.4815, train_loss: 0.21757787, train_accuracy: 0.9453, test_Accuracy: 0.9509
Epoch: [ 0] [  299/  468] time: 28.5740, train_loss: 0.20907217, train_accuracy: 0.9297, test_Accuracy: 0.9508
Epoch: [ 0] [  300/  468] time: 28.6668, train_loss: 0.25615394, train_accuracy: 0.9297, test_Accuracy: 0.9519
Epoch: [ 0] [  301/  468] time: 28.7611, train_loss: 0.10746391, train_accuracy: 0.9766, test_Accuracy: 0.9525
Epoch: [ 0] [  302/  468] time: 28.8576, train_loss: 0.09740644, train_accuracy: 0.9766, test_Accuracy: 0.9543
Epoch: [ 0] [  303/  468] time: 28.9526, train_loss: 0.10737349, train_accuracy: 0.9766, test_Accuracy: 0.9543
Epoch: [ 0] [  304/  468] time: 29.0455, train_loss: 0.05210468, train_accuracy: 1.0000, test_Accuracy: 0.9549
Epoch: [ 0] [  305/  468] time: 29.1387, train_loss: 0.15345418, train_accuracy: 0.9375, test_Accuracy: 0.9553
E

Epoch: [ 0] [  373/  468] time: 35.6519, train_loss: 0.15533739, train_accuracy: 0.9453, test_Accuracy: 0.9559
Epoch: [ 0] [  374/  468] time: 35.7475, train_loss: 0.13455072, train_accuracy: 0.9688, test_Accuracy: 0.9572
Epoch: [ 0] [  375/  468] time: 35.8445, train_loss: 0.19740234, train_accuracy: 0.9609, test_Accuracy: 0.9579
Epoch: [ 0] [  376/  468] time: 35.9392, train_loss: 0.05739752, train_accuracy: 0.9844, test_Accuracy: 0.9585
Epoch: [ 0] [  377/  468] time: 36.0372, train_loss: 0.14384778, train_accuracy: 0.9688, test_Accuracy: 0.9581
Epoch: [ 0] [  378/  468] time: 36.1375, train_loss: 0.17199636, train_accuracy: 0.9531, test_Accuracy: 0.9565
Epoch: [ 0] [  379/  468] time: 36.2401, train_loss: 0.20914838, train_accuracy: 0.9375, test_Accuracy: 0.9547
Epoch: [ 0] [  380/  468] time: 36.3364, train_loss: 0.12735286, train_accuracy: 0.9453, test_Accuracy: 0.9527
Epoch: [ 0] [  381/  468] time: 36.4384, train_loss: 0.10665734, train_accuracy: 0.9609, test_Accuracy: 0.9524
E

Epoch: [ 0] [  448/  468] time: 42.9204, train_loss: 0.12465809, train_accuracy: 0.9688, test_Accuracy: 0.9614
Epoch: [ 0] [  449/  468] time: 43.0195, train_loss: 0.12483499, train_accuracy: 0.9609, test_Accuracy: 0.9631
Epoch: [ 0] [  450/  468] time: 43.1135, train_loss: 0.09034453, train_accuracy: 0.9688, test_Accuracy: 0.9639
Epoch: [ 0] [  451/  468] time: 43.2109, train_loss: 0.12152860, train_accuracy: 0.9688, test_Accuracy: 0.9646
Epoch: [ 0] [  452/  468] time: 43.3006, train_loss: 0.05697597, train_accuracy: 0.9766, test_Accuracy: 0.9649
Epoch: [ 0] [  453/  468] time: 43.3974, train_loss: 0.08105034, train_accuracy: 0.9844, test_Accuracy: 0.9644
Epoch: [ 0] [  454/  468] time: 43.4874, train_loss: 0.14425385, train_accuracy: 0.9766, test_Accuracy: 0.9645
Epoch: [ 0] [  455/  468] time: 43.5841, train_loss: 0.07521962, train_accuracy: 0.9766, test_Accuracy: 0.9643
Epoch: [ 0] [  456/  468] time: 43.6834, train_loss: 0.07860439, train_accuracy: 0.9688, test_Accuracy: 0.9647
E

# Test accuracy : 96.55 %